In [1]:
import os
os.chdir("../")

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model:Path
    training_data: Path
    all_params:dict
    params_image_size: list
    params_batch_size:int

In [5]:
from cnnClassifer.constants import * 
from cnnClassifer.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Vegetable Images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [7]:
from urllib.parse import urlparse

In [15]:
from cnnClassifer.utils.common import save_json


class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config =config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
        

    def evaluation(self):
        model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2023-10-12 20:08:49,142: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-12 20:08:49,145: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-12 20:08:49,146: INFO: common: created directory at : artifacts]
Found 6300 images belonging to 15 classes.
394/394 [==============================] - 65s 147ms/step - loss: 0.3225 - accuracy: 0.9184
[2023-10-12 20:09:55,305: INFO: common: json file saved at: scores.json]


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]